In [1]:
from __future__ import print_function, division

import requests
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import re 
from dateutil.parser import parse
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pickle

%matplotlib inline

/Users/liyuqiao/anaconda/envs/py3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Load Data

In [2]:
with open('../../Data/movies.pickle', 'rb') as f:
    df = pickle.load(f)
    
df.Actors = df.Actors.apply(lambda x: ', '.join(x))

### Data Preprocessing: 
* Data type conversion  
* Time stamp conversion   
* Handling missing values 
* Getting rid of duplicates 

In [6]:
# data type conversions 
df['Directors'] = df['Directors'].astype('str')
df['Release Year'] = df['Release Year'].astype('str')

def theathers_to_int(i):
    if type(i) != int:
        return int(i.replace(',',''))
    else: 
        return i 
df['Wildest Release'] = df['Wildest Release'].apply(theathers_to_int) 

def check_lst(x):
    if type(x) == list:
        x = ', '.join(x)
    else: 
        x = x
    return x 
df['Directors'] = df['Directors'].apply(check_lst)

In [ ]:
# drop movies released before 2007 
df = df[df['Release Date'].dt.year >= 2007]

In [14]:
# replace missing values with NaN  
df = df.replace('N/A', np.NaN)
df = df.replace(['N/A'], np.NaN)
df = df.replace(['N/a'], np.NaN)

# fill in the null values for Domestic Total Gross with mean 
df = df.fillna({"Domestic Total Gross ($)": 
                df["Domestic Total Gross ($)"].mean()})

# fill in the null values for Domestic Total Gross with median
df = df.fillna({"Production Budget ($)": 
                df["Production Budget ($)"].median()})

# find the mode for 'Runtime' 
df['Runtime (mins)'].value_counts()

# fill in null values for Runtime with mode 
df = df.fillna({'Runtime (mins)': float(95)})

In [20]:
# drop duplicated values 
df = df.drop_duplicates(subset='Movie Title',keep='first')

In [22]:
# time stamp conversion 
df['Release Year'] = df['Release Date'].apply(lambda x: x.year)
df['Release Date'] = df['Release Date'].apply(lambda x: x.month)

In [31]:
# take inflation into consideration for gross computation 
def year_inflation(year):
    if year == '2007': 
        return 0.843
    elif year == '2008': 
        return 0.875
    elif year == '2009':
        return 0.872
    elif year == '2010':
        return 0.886
    elif year == '2011': 
        return 0.914
    elif year == '2012': 
        return 0.933
    elif year == '2013':
        return 0.947
    elif year == '2014': 
        return 0.962
    elif year == '2015': 
        return 0.963
    elif year == '2016': 
        return 0.976
    else:
        return 1
df['Release Year'] = df['Release Year'].apply(year_inflation)

# reflect inflation on monetary columns 
df['Production Budget ($)'] = (df['Production Budget ($)'] 
                                      / df['Release Year'])

df['Opening Weekend Gross ($)'] = (df['Opening Weekend Gross ($)'] 
                                      / df['Release Year'])

df['Domestic Total Gross ($)'] = (df['Domestic Total Gross ($)'] 
                                      / df['Release Year'])

df['Opening Weekend Gross ($)']=[x/1000000 for 
                                 x in df['Opening Weekend Gross ($)']]
df['DomesticTotalGross']=[x/1000000 for x in df['DomesticTotalGross']]

In [39]:
# Other conversions -standardize names 
df.rename(columns={'Release Date':'Release Month'}, inplace=True)
df.rename(columns={'Domestic Total Gross ($)':'DomesticTotalGross'},
          inplace=True)

### Dummify Categorical Features Into Different Groups

In [42]:
# Release Month 
df = pd.get_dummies(df, columns=['Release Month'], 
                    prefix=["Release Month"], 
                    drop_first=True)

In [43]:
# Ratings
def ratings_buckets(rating):
    if rating == 'PG-13' or rating == 'PG':
        return 'PG/PG-13'
    elif rating == 'Unknown':
        return 'Unrated'
    else: 
        return rating 
df['Rating'] = df['Rating'].apply(ratings_buckets)

df = pd.get_dummies(df, columns=['Rating'], 
                    prefix=["MPAA"], 
                    drop_first=True)

df = pd.get_dummies(df, columns=['Genre'], 
                    prefix=["Genre"], 
                    drop_first=True)

In [53]:
# Production Budget 
budget_mean = df['Production Budget ($)'].mean()
df.drop('Release Year', axis=1, inplace = True)
[item.get_ydata() for item in B['whiskers']]
interquartile =[item.get_ydata()[0] for item in B['whiskers']]
twenty_five_percentile = interquartile[0]
seventy_five_percentile = interquartile[1]

def prod_budget_buckets(budget):
    if budget < twenty_five_percentile:
        return 'Small'
    elif budget < seventy_five_percentile:
        return 'Medium'
    else: 
        return 'Large'
    
df['Production Budget ($)'] = df['Production Budget ($)'].apply(
    prod_budget_buckets)

df = pd.get_dummies(df, 
                    columns=['Production Budget ($)'], 
                    prefix = 'Prod_Budget', 
                    drop_first=True)

In [56]:
# Release Theaters 
def wild_release_buckets(theaters):
    if theaters <= 599:
        return 'Limited'
    elif theaters <= 3000:
        return 'Wild'
    else: 
        return 'Huge'
    
df['Wildest Release'] = df['Wildest Release'].apply(wild_release_buckets)

df = pd.get_dummies(df, 
                    columns=['Wildest Release'], 
                    prefix = 'Release', 
                    drop_first=True)

#### Information on Oscar winners for Best Actor/Actress/Director
http://www.nndb.com

In [60]:
# Depending on director's average domestic total gross per movie, 
# bag directors  
df.groupby(['Directors']).DomesticTotalGross.mean().sort_values(
    ascending=False)[:10]

Directors
James Cameron                   7.497661e+08
Colin Trevorrow                 6.522706e+08
Joss Whedon                     5.411819e+08
Bill Condon                     5.040142e+08
Angus MacLane Andrew Stanton    4.862956e+08
J.J. Abrams                     4.308150e+08
Lee Unkrich                     4.150049e+08
Patty Jenkins                   4.120388e+08
Shane Black                     4.090140e+08
Gary Ross                       4.080107e+08
Name: DomesticTotalGross, dtype: float64

In [61]:
def director_gross_ranking(director):
    if (df.groupby(['Directors'])
        .DomesticTotalGross.mean()
        .sort_values(ascending=False)[director]) > 400000000:
        return 'Above_40'
    elif (df.groupby(['Directors'])
        .DomesticTotalGross.mean()
        .sort_values(ascending=False)[director]) > 200000000:
        return 'Above_20'
    elif (df.groupby(['Directors'])
        .DomesticTotalGross.mean()
        .sort_values(ascending=False)[director]) > 99999999:
        return 'Above_10'
    elif (df.groupby(['Directors'])
        .DomesticTotalGross.mean()
        .sort_values(ascending=False)[director]) > 9999999:
        return 'Above_1'
    else: 
        return 'Below_1'
    
df['Directors'] = df['Directors'].astype('str')
df['Directors'] = df['Directors'].apply(director_gross_ranking)

df = pd.get_dummies(df, 
                    columns=['Directors'], 
                    prefix = 'Director', 
                    drop_first=True)

In [65]:
# Actors 
valuable_actors_df = pd.read_csv('valuable_actors.txt', sep=",")
valuable_actors_df.columns = ['index','actor/actress'
                     ,'total gross','movies'
                     ,'avg gross','#1 picture','gross']

valuable_actors_df = valuable_actors_df['actor/actress']

In [68]:
def check_actors_match(actors):
    """Returns a counter that helps transform star power
    to measurable values. Any actor/actress that are within 
    the top 100 highest total lifetime gross list will be 
    given a score of 2, else 1 (0 for NaN values).
     """
    count = 1
    for name in valuable_actors_df:
        if name in actors:
            count += 1  
    return count 

df['Actors'] = df['Actors'].astype('str')
df['Actors'] = df['Actors'].apply(lambda x : x.replace(', ',' '))
df['Actors'] = df['Actors'].apply(check_actors_match)

df = pd.get_dummies(df, 
                    columns=['Actors'], 
                    prefix = 'Actors Rank', 
                    drop_first=True)